In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#Load dữ liệu
df = pd.read_csv('data.csv', header=None)
display(df)

,0,1,2,3,4,5
0,0,1,2,3,4,5
1,Wine,Chips,Bread,Butter,Milk,Apple
2,Wine,Chips,Bread,Butter,Milk,NaN
3,Wine,NaN,Bread,Butter,Milk,NaN
4,Wine,Chips,NaN,NaN,NaN,Apple
5,Wine,Chips,Bread,Butter,Milk,Apple
6,Wine,Chips,Bread,Butter,Milk,NaN
7,Wine,Chips,Bread,Butter,Milk,Apple
8,Wine,NaN,Bread,Butter,Milk,Apple
9,Wine,Chips,NaN,NaN,Milk,Apple


In [5]:
records = []
for i in range(0, df.shape[0]):
    records.append([str(df.values[i, j]) for j in range(0, df.shape[1])])

In [7]:
#chuyển records thành transaction
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
display(df1)

,0,1,2,3,4,5,Apple,Bread,Butter,Chips,Milk,Wine,nan
0,True,True,True,True,True,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,True,True,True,True,True,False
2,False,False,False,False,False,False,False,True,True,True,True,True,True
3,False,False,False,False,False,False,False,True,True,False,True,True,True
4,False,False,False,False,False,False,True,False,False,True,False,True,True
5,False,False,False,False,False,False,True,True,True,True,True,True,False
6,False,False,False,False,False,False,False,True,True,True,True,True,True
7,False,False,False,False,False,False,True,True,True,True,True,True,False
8,False,False,False,False,False,False,True,True,True,False,True,True,True
9,False,False,False,False,False,False,True,False,False,True,True,True,True


In [8]:
frequent_itemsets = apriori(df1, min_support=0.6, use_colnames=True)
display(frequent_itemsets)

,support,itemsets
0,0.695652,(Apple)
1,0.826087,(Bread)
2,0.782609,(Butter)
3,0.826087,(Chips)
4,0.826087,(Milk)
5,0.956522,(Wine)
6,0.608696,"(Bread, Apple)"
7,0.608696,"(Chips, Apple)"
8,0.608696,"(Milk, Apple)"
9,0.695652,"(Apple, Wine)"


In [9]:
# build association rules using support metric
rules = association_rules(frequent_itemsets, metric="support", support_only=True, 
                          min_threshold=0.1)

rules = rules[['antecedents', 'consequents', 'support']]
print(rules)

    antecedents                   consequents   support
0       (Bread)                       (Apple)  0.608696
1       (Apple)                       (Bread)  0.608696
2       (Chips)                       (Apple)  0.608696
3       (Apple)                       (Chips)  0.608696
4        (Milk)                       (Apple)  0.608696
..          ...                           ...       ...
201     (Chips)   (Milk, Wine, Bread, Butter)  0.608696
202     (Bread)   (Milk, Chips, Wine, Butter)  0.608696
203      (Milk)  (Wine, Chips, Bread, Butter)  0.608696
204    (Butter)    (Milk, Chips, Bread, Wine)  0.608696
205      (Wine)  (Milk, Chips, Bread, Butter)  0.608696

[206 rows x 3 columns]
